In [1]:
import numpy as np
import sys
import os
#import tensorflow as tf

import io
from gensim.models import word2vec

from io import StringIO
from pprint import pprint

from nltk.tokenize import word_tokenize
import nltk

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import types
from IPython import get_ipython #for import notebook
from nbformat import read #for import notebook
from IPython.core.interactiveshell import InteractiveShell #for import notebook

In [3]:
def find_notebook(fullname, path=None): #for import notebook
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [4]:
class NotebookLoader(object): #for import notebook
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [5]:
sys.meta_path.append(NotebookFinder())

In [6]:
import torch
import torch.nn as nn
from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.utils.data
from torchvision import transforms, utils
from torch.utils.data import DataLoader

In [7]:
import pre_process

importing Jupyter notebook from pre_process.ipynb

 [[1561, 1, 1089, 1, 102, 33, 74, 14, 7, 229, 3510, 218, 354, 4], [32, 39, 16, 10, 4904, 43, 10, 68, 44, 60, 14, 81, 2579, 0], [29, 21, 3, 225, 4, 41, 35, 93, 149, 6, 1314, 0], [71, 3, 68, 38, 40, 4, 2, 64, 0, 41, 35, 51, 106, 149, 1473, 8, 2582, 1735, 0, 1409, 177, 61, 4], [2, 325, 3, 19, 6800, 57, 654, 30, 21, 4, 2, 64, 207, 24, 1492, 36, 224, 0], [2, 746, 7, 547, 144, 6, 5, 1215, 230, 30, 23, 348, 4963, 8, 273, 56, 12, 81, 358, 0], [97, 7, 60, 228, 0, 2, 318, 476, 8, 2153, 364, 47, 46, 6, 348, 'pingpong.Perhaps', 30, 23, 106, 7, 'foursome', 28, 99, 0], [609, 133, 6, 20, 17, 140, 88, 19, 1180, 1, 30, 121, 250, 99, 6, 47, 1383, 28, 'us.That', 10, 771, 645, 8, 449, 1, 78, 0], ['Good.Let', 161, 34, 47, 92, 0], [219, 55, 0], []]

 [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1,

In [8]:
my_dataset = pre_process.transformed_train_dataset

In [9]:
len(my_dataset)

11118

In [10]:
class chr_rnn(nn.Module):
    def __init__(self, hidden_size, bidirectional):
        super(chr_rnn, self).__init__()
        self.hidden_size = hidden_size
        if(bidirectional == True):
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.chr_rnn = nn.GRU(100, 100, bidirectional = bidirectional)
        
        
    def forward(self,char,h0):
        _,h0 = self.chr_rnn(char,h0)
        return h0
        
    def initHidden(self):
        return torch.zeros(self.bidirectional , 1, self.hidden_size, device=device)
    
    
    
    
class last_net(nn.Module):
    def __init__(self):
        super(last_net, self).__init__()
        self.lastnet = nn.Linear(200,100)
        
    def forward(self,last_hidden_state):
        last_w = self.lastnet(last_hidden_state)
        return last_w
  
    
chr_rnn_1 = chr_rnn(100, True).cuda()
last_net_1 = last_net().cuda()


In [11]:
if os.path.isfile('/home/jongsu/Desktop/persona_dialogue/chr_parameter/my_character_rnn_526.pth'):
    print("loading chr")
    chr_rnn_1.load_state_dict(torch.load('/home/jongsu/Desktop/persona_dialogue/chr_parameter/my_character_rnn_526.pth'))

if os.path.isfile('/home/jongsu/Desktop/persona_dialogue/chr_parameter/my_character_linear_526.pth'):
    print("loading last")
    last_net_1.load_state_dict(torch.load('/home/jongsu/Desktop/persona_dialogue/chr_parameter/my_character_linear_526.pth'))

In [12]:
wv_model_en = word2vec.Word2Vec(size=100, window=5, min_count=5, workers=4)
wv_model_en = word2vec.Word2Vec.load('/home/jongsu/Desktop/intention_extraction/wv_parameter/dialogue_wv')

In [13]:
class makesent_gru(nn.Module):
    def __init__(self, hidden_size, bidirectional):
        super(makesent_gru, self).__init__()
        self.hidden_size = hidden_size
        if(bidirectional == True):
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.gru = nn.GRU(100, 100, bidirectional = bidirectional)
        
        
    def forward(self,char,h0):
        gru,h0 = self.gru(char,h0)
        return h0
        
    def initHidden(self):
        return torch.zeros(self.bidirectional , 1, self.hidden_size, device=device)
 

In [14]:
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


In [15]:
criterian = nn.CrossEntropyLoss()


class Gru(nn.Module):
    
    def __init__(self, hidden_dim):
        super(Gru, self).__init__()
        self.gru = nn.GRU(100,100,bidirectional = True)
        self.inh = torch.zeros(2, 1, 100).cuda()
        self.nn = nn.Linear(200,28)

        self.softmax = nn.Softmax(dim = 0)
        
    def init_hidden(self):
        return torch.zeros(2, 1, 100).cuda()
    
    def Loss(self, output, target):
        i = 0
        loss = 0
        while(i < len(output)):
            
            j = 0
            while(j < len(output[i])):
                
                if (j == target[i]):
                    
                    
                    loss = loss + 0
                else:
                    loss = loss + torch.log(output[i][j])
                
                j = j + 1
            
            target
            i = i + 1
            
        return -loss
            
    
    def forward(self, sentence):  
        
      
        
        output,hidden = self.gru(sentence,self.inh)
        
        output = self.nn(output.view(len(sentence),200))
        
        output2 = self.softmax(output)
        
        return output2

In [16]:
class last_sent_net(nn.Module):
    def __init__(self):
        super(last_sent_net, self).__init__()
        self.lastnet = nn.Linear(200,100)
        
    def forward(self,last_hidden_state):
        last_w = self.lastnet(last_hidden_state)
        return last_w


In [17]:
tagset = torch.tensor(np.arange(28),device=device)

In [18]:
sent_1 = makesent_gru(100,True).cuda()
last_sent_1 = last_sent_net().cuda()
Gru1 = Gru(100).cuda()

In [19]:
'''
if os.path.isfile('/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/Bigru_1.pth'):
    print("loading crf")
    Gru1.load_state_dict(torch.load('/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/Bigru_1.pth'))

if os.path.isfile('/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/last_sent_1.pth'):
    print("loading last")
    last_sent_1.load_state_dict(torch.load('/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/last_sent_1.pth'))
    
if os.path.isfile('/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/sent_1.pth'):
    print("loading last")
    sent_1.load_state_dict(torch.load('/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/sent_1.pth'))
'''

'\nif os.path.isfile(\'/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/Bigru_1.pth\'):\n    print("loading crf")\n    Gru1.load_state_dict(torch.load(\'/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/Bigru_1.pth\'))\n\nif os.path.isfile(\'/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/last_sent_1.pth\'):\n    print("loading last")\n    last_sent_1.load_state_dict(torch.load(\'/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/last_sent_1.pth\'))\n    \nif os.path.isfile(\'/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/sent_1.pth\'):\n    print("loading last")\n    sent_1.load_state_dict(torch.load(\'/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/sent_1.pth\'))\n'

In [20]:
a = [1,2,3,4]

In [21]:
a[1:4]

[2, 3, 4]

In [26]:
lens = range(10)

In [32]:
lens[9]

9

In [34]:
lens[::-1][9]

0

In [43]:
lens = np.arange(10)
lens + 1

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [51]:

x = Variable(torch.randn(10, 20, 30)).cuda()
print("x",x)
print("x",x[0,:])
lens = np.arange(10) + 1

x = pack_padded_sequence(x, lens[::-1], batch_first=True)


x tensor([[[ 0.7180, -0.1102,  0.6078,  ..., -1.3413,  0.6959,  1.5664],
         [-1.8044, -0.8018, -0.8930,  ...,  0.4830, -2.0441, -0.4130],
         [ 0.7650,  0.2513,  1.0740,  ..., -0.5877, -0.2748, -1.7118],
         ...,
         [ 1.2101,  0.1563, -2.5640,  ...,  0.5213, -0.4187, -0.2883],
         [-0.0119,  0.1858,  0.0913,  ...,  0.9516, -0.7111, -0.8005],
         [ 0.0893, -0.2594, -0.1451,  ...,  2.0239, -0.6212,  0.2858]],

        [[-0.3570,  0.3399,  0.2410,  ..., -1.0136, -1.2953, -0.3620],
         [-0.5911,  0.1988, -0.4088,  ...,  1.3073, -0.5890, -0.9550],
         [-1.0861, -0.3116, -0.5129,  ..., -0.6419,  0.2673, -1.4841],
         ...,
         [-0.5738,  0.2410,  1.7730,  ..., -1.2751,  0.9195,  1.3656],
         [ 0.4975,  0.1703, -1.1401,  ..., -0.9224,  0.1239, -0.1094],
         [ 1.3168, -1.2167, -0.7636,  ...,  0.4232,  0.9514,  0.4807]],

        [[ 1.3790,  0.9043, -1.6484,  ...,  1.7026, -0.0262,  1.0267],
         [-1.0163, -0.5424, -1.1637,  ..., 

In [46]:
x

PackedSequence(data=tensor([[ 0.0856,  0.0644, -0.3780,  ..., -0.3990, -0.3624,  0.1106],
        [-0.7543,  0.2678,  0.9364,  ...,  0.1133,  1.2389, -0.8259],
        [ 0.6406,  0.5313,  0.6551,  ...,  1.2418,  0.0284, -0.0325],
        ...,
        [-0.4823, -0.9832,  0.1907,  ...,  0.5451, -1.3907,  0.8209],
        [ 0.0749, -0.5988, -0.1443,  ...,  0.7389,  0.9940, -1.5226],
        [ 1.2861, -0.9599, -0.0999,  ...,  0.1092,  1.5392, -0.4489]], device='cuda:0'), batch_sizes=tensor([ 10,   9,   8,   7,   6,   5,   4,   3,   2,   1]))

In [44]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

x = Variable(torch.randn(10, 20, 30)).cuda()
lens = np.arange(10) + 1

x = pack_padded_sequence(x, lens[::-1], batch_first=True)

lstm = nn.LSTM(30, 50, batch_first=True).cuda()
h0 = Variable(torch.zeros(1, 10, 50)).cuda()
c0 = Variable(torch.zeros(1, 10, 50)).cuda()

packed_h, (packed_h_t, packed_c_t) = lstm(x, (h0, c0))
h, _ = pad_packed_sequence(packed_h) 
print (h.size()) # Size 20 x 10 x 50 instead of 10 x 20 x 50

torch.Size([10, 10, 50])


In [35]:
optimizer1 = optim.SGD(Gru1.parameters(), lr=0.0001)#, weight_decay=1e-4)
optimizer2 = optim.SGD(sent_1.parameters(), lr=0.0001)#, weight_decay=1e-4)
optimizer3 = optim.SGD(last_sent_1.parameters(), lr=0.0001)#, weight_decay=1e-4)

epoch = 0

while(epoch < 100): 
    
    total_lss = 0
    dialogue_idx = 1
    while(dialogue_idx =< len(my_dataset)/100 ):
        
        if(dialogue_idx == 1):
            batch_dialogue = my_dataset[0:100]
        else:
            batch_dialogue = my_dataset[dialogue_idx-100:dialogue_idx] #make batch
            
            
        sent_idx = 0
        new_sent_to_vec = []
        sent_to_vec = torch.tensor(new_sent_to_vec).cuda()
        
        
        make_label = []
        new_label = np.array([])
        while(sent_idx < len(per_dialogue['dialogue'][0]) -1): #indexed dialogue
            
            ## sent_lstm training
            sent = per_dialogue['dialogue'][0][sent_idx]
            sent_len = len(per_dialogue['dialogue'][0][sent_idx]) 
            
            word_idx = 0
            vec = []
            
            
            while(word_idx < sent_len): 
                
                
                encoder_hidden = chr_rnn_1.initHidden()
                if per_dialogue['dialogue'][1][sent_idx][word_idx] == 1: #word_to_vec
                    word_to_tensor = torch.tensor(wv_model_en.wv[wv_model_en.wv.index2entity[sent[word_idx]]])
                    gpu_wt = word_to_tensor.cuda()
                    vec.append(gpu_wt) #append known word vec
                else: #character model
                    for ei in range(len(sent[word_idx])):
                        gru_input = torch.tensor(np.zeros(shape = (1, 1, 100), dtype="float32")).cuda()
                        gru_input[0][0][ord(sent[word_idx][ei])%100] = 1 #character to onehot vec
                        encoder_hidden = chr_rnn_1(gru_input,encoder_hidden)

                    last_encoder_hidden = encoder_hidden.view(-1,200)
                    last_encoder_hidden_ = last_net_1(last_encoder_hidden) #last hiddenstate and fully c l
                    vec.append(last_encoder_hidden_) #append unknown word vec
                word_idx = word_idx + 1
            
            
            word_idx = 0 
            
            sent_hidden = sent_1.initHidden()
            
            
            while(word_idx<sent_len):

                view = vec[word_idx].view(1,1,100)
                sent_hidden = sent_1(view,sent_hidden)
                word_idx = word_idx + 1
            
            # 200 -> 100
            
            sent_vec = last_sent_1(sent_hidden.view(-1,200))
   
            sent_to_vec = torch.cat((sent_to_vec,sent_vec),0)
            

            index1_toint = int(per_dialogue['label'][0][sent_idx*2]) *4
            index2_toint = int(per_dialogue['label'][1][sent_idx*2])
            
        
            new_label = np.append(new_label,index1_toint + index2_toint -1) #hot first index,, index1 prob
            
            sent_idx = sent_idx + 1
        
        
        resh_stv = sent_to_vec.view(-1,1,100).cuda()
        
        torch_make_label = torch.LongTensor(new_label).cuda()

        resh_stv = Gru1(resh_stv)
        
        loss = Gru1.Loss(resh_stv, torch_make_label) #-> 
        
        
        
 
        ## lstm_crf training

        dialogue_idx = dialogue_idx + 1
        
        total_lss = total_lss + loss
        
        if(dialogue_idx % 1 ==0):
            print("cal grad..")
            Gru1.zero_grad()
            sent_1.zero_grad()
            last_sent_1.zero_grad()
            
            total_lss.backward()
            print(total_lss)
            
            optimizer1.step()
            optimizer2.step()
            optimizer3.step()
            
            total_lss = 0
            
            torch.save(Gru1.state_dict(),'/home/jongsu/Desktop/intention_extraction/crf_lstm_parameter/Bigru_2.ckpt')
            torch.save(sent_1.state_dict(),'/home/jongsu/Desktop/intention_extraction/crf_lstm_parameter/sent_2.ckpt')
            torch.save(last_sent_1.state_dict(),'/home/jongsu/Desktop/intention_extraction/crf_lstm_parameter/last_sent_2.ckpt')

            torch.save(Gru1.state_dict(),'/home/jongsu/Desktop/intention_extraction/crf_lstm_parameter/Bigru_2.pkl')
            torch.save(sent_1.state_dict(),'/home/jongsu/Desktop/intention_extraction/crf_lstm_parameter/sent_2.pkl')
            torch.save(last_sent_1.state_dict(),'/home/jongsu/Desktop/intention_extraction/crf_lstm_parameter/last_sent_2.pkl')

            torch.save(Gru1.state_dict(),'/home/jongsu/Desktop/intention_extraction/crf_lstm_parameter/Bigru_2.pth')
            torch.save(sent_1.state_dict(),'/home/jongsu/Desktop/intention_extraction/crf_lstm_parameter/sent_2.pth')
            torch.save(last_sent_1.state_dict(),'/home/jongsu/Desktop/intention_extraction/crf_lstm_parameter/last_sent_2.pth')
        
    epoch = epoch + 1

cal grad..
tensor(621.7704, device='cuda:0')
cal grad..
tensor(621.7701, device='cuda:0')
cal grad..
tensor(621.7701, device='cuda:0')
cal grad..
tensor(621.7698, device='cuda:0')
cal grad..
tensor(621.7693, device='cuda:0')
cal grad..
tensor(621.7690, device='cuda:0')
cal grad..
tensor(621.7689, device='cuda:0')
cal grad..
tensor(621.7684, device='cuda:0')
cal grad..
tensor(621.7680, device='cuda:0')
cal grad..
tensor(621.7680, device='cuda:0')
cal grad..
tensor(621.7676, device='cuda:0')
cal grad..
tensor(621.7672, device='cuda:0')
cal grad..
tensor(621.7668, device='cuda:0')
cal grad..
tensor(621.7666, device='cuda:0')
cal grad..
tensor(621.7662, device='cuda:0')
cal grad..
tensor(621.7660, device='cuda:0')
cal grad..
tensor(621.7657, device='cuda:0')
cal grad..
tensor(621.7657, device='cuda:0')
cal grad..
tensor(621.7655, device='cuda:0')
cal grad..
tensor(621.7648, device='cuda:0')
cal grad..
tensor(621.7648, device='cuda:0')
cal grad..
tensor(621.7645, device='cuda:0')
cal grad..

KeyboardInterrupt: 

In [ ]:
np.zeros(10)